In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
from tqdm import tqdm, trange


['ner.csv', 'ner_dataset.csv']


In [49]:
input_data = pd.read_csv("../input/ner_dataset.csv", encoding="latin1")
#input_data

In [50]:
input_data = input_data.fillna(method="ffill")
input_data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [51]:
words_list = list(set(input_data["Word"].values))
words_list[:10]

['interviewer',
 'spiritual',
 'immunizations',
 'beating',
 'lslamic',
 'Zedong',
 'respecting',
 '10-to-eight',
 'rebuke',
 'starlet']

In [5]:
number_words = len(words_list); number_words

35178

In [7]:
class RetrieveSentance(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        function = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(function)
        self.sentences = [s for s in self.grouped]
    
    def retrieve(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [8]:
Sentences = RetrieveSentance(input_data)
Sentences

In [9]:
Sentences_list = [" ".join([s[0] for s in sent]) for sent in Sentences.sentences]
Sentences_list[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [10]:
labels = [[s[2] for s in sent] for sent in Sentences.sentences]
print(labels[0])

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [12]:
tags2vals = list(set(input_data["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tags2vals)}

In [15]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [16]:
max_seq_len = 75 # tokens
batch_s = 32 # batch size

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [18]:
torch.cuda.get_device_name(0) 

'Tesla P100-PCIE-16GB'

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=True)


100%|██████████| 213450/213450 [00:00<00:00, 1093181.60B/s]


In [21]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in Sentences_list]
print(tokenized_texts[0])

['thousands', 'of', 'demons', '##tra', '##tors', 'have', 'marched', 'through', 'lo', '##ndon', 'to', 'protest', 'the', 'war', 'in', 'i', '##ra', '##q', 'and', 'demand', 'the', 'withdrawal', 'of', 'br', '##itis', '##h', 'troops', 'from', 'that', 'country', '.']


In [26]:
X = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_seq_len, dtype="long", truncating="post", padding="post")

In [27]:
Y = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=max_seq_len, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [28]:
X

array([[ 4674,  1104,  8568, ...,     0,     0,     0],
       [  178, 23851,  1389, ...,     0,     0,     0],
       [ 7948,  3832, 12526, ...,     0,     0,     0],
       ...,
       [ 1378,   178,  4047, ...,     0,     0,     0],
       [ 1290,  1173,   117, ...,     0,     0,     0],
       [ 1103, 10280,  6015, ...,     0,     0,     0]])

In [29]:
Y

array([[15, 15, 15, ..., 15, 15, 15],
       [ 6, 15, 15, ..., 15, 15, 15],
       [15, 15, 11, ..., 15, 15, 15],
       ...,
       [15,  7, 15, ..., 15, 15, 15],
       [15, 15, 15, ..., 15, 15, 15],
       [15, 16,  5, ..., 15, 15, 15]])

In [30]:
attention_masks = [[float(i>0) for i in ii] for ii in X]

In [32]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, 
                                                            random_state=20, test_size=0.1)
Mask_train, Mask_valid, _, _ = train_test_split(attention_masks, X,
                                             random_state=20, test_size=0.1)

In [33]:
X_train = torch.tensor(X_train)
X_valid = torch.tensor(X_valid)
Y_train = torch.tensor(Y_train)
Y_valid = torch.tensor(Y_valid)
Mask_train = torch.tensor(Mask_train)
Mask_valid = torch.tensor(Mask_valid)

In [35]:
data_train = TensorDataset(X_train, Mask_train, Y_train)
data_train_sampler = RandomSampler(data_train)
DL_train = DataLoader(data_train, sampler=data_train_sampler, batch_size=batch_s)

data_valid = TensorDataset(X_valid, Mask_valid, Y_valid)
data_valid_sampler = SequentialSampler(data_valid)
DL_valid = DataLoader(data_valid, sampler=data_valid_sampler, batch_size=batch_s)

In [36]:
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(tag2idx))

100%|██████████| 404400730/404400730 [00:11<00:00, 35819407.19B/s]


In [52]:
model.cuda();

In [39]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [42]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [46]:
epochs = 5
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(DL_train):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in DL_valid:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags2vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags2vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))




Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.04349928928071079
Validation loss: 0.04484805369128784
Validation Accuracy: 0.9859369047619049





Epoch:  20%|██        | 1/5 [07:29<29:56, 449.19s/it]

F1-Score: 0.7386971097218448
Train loss: 0.03685836643021998
Validation loss: 0.04377520118529598
Validation Accuracy: 0.9870265873015874





Epoch:  40%|████      | 2/5 [14:58<22:27, 449.25s/it]

...

F1-Score: 0.7504563465562532
Train loss: 0.027583255931345665
Validation loss: 0.04901061260451873
Validation Accuracy: 0.9862666666666668





Epoch:  80%|████████  | 4/5 [29:57<07:29, 449.37s/it]

F1-Score: 0.7546185803942367
Train loss: 0.02427265120206264
Validation loss: 0.04626476022104422
Validation Accuracy: 0.9877718253968256





Epoch: 100%|██████████| 5/5 [37:26<00:00, 449.33s/it]

F1-Score: 0.7769945674134602


In [48]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in DL_valid:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags2vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags2vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Validation loss: 0.04626476022104422
Validation Accuracy: 0.9877718253968256
Validation F1-Score: 0.7769945674134602
